In [2]:
#导入一系列包
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
file_path2='D:/files/data/stopwordlisteng.TXT'
stopworddic1=stop_words
data=[]
with open(file_path2, "r") as f:
    for line in f.readlines():
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        data.append(line)
        #print(line)
for i in range(len(data)):
    stopworddic1.append(data[i])

fh=[',','(',')','e','.','g','-']

for i in range(1000):
    fh.append(str(i))

for i in range(len(fh)):
    stopworddic1.append(fh[i])

In [5]:
# Import Dataset
file_path='D:/files/data/wos/result4-1.xls'
df=pd.read_excel(file_path)

In [6]:
data = df.keycontent.values.tolist()

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['total', 'carbon', 'emissions', 'minimization', 'in', 'connected', 'and', 'automated', 'vehicle', 'routing', 'problem', 'with', 'speed', 'variables', 'environmental', 'protection', 'and', 'intelligence', 'have', 'become', 'the', 'inevitable', 'development', 'trend', 'of', 'future', 'transportation', 'connected', 'and', 'automated', 'vehicles', 'cavs', 'are', 'expected', 'to', 'be', 'applied', 'in', 'the', 'near', 'future', 'in', 'this', 'context', 'how', 'to', 'schedule', 'cavs', 'to', 'meet', 'customer', 'demands', 'with', 'carbon', 'emissions', 'minimization', 'has', 'become', 'new', 'green', 'vehicle', 'routing', 'problem', 'vrp', 'due', 'to', 'the', 'fact', 'that', 'carbon', 'emissions', 'are', 'tremendously', 'influenced', 'by', 'vehicle', 'speed', 'this', 'paper', 'considers', 'vehicle', 'speed', 'as', 'decision', 'variable', 'in', 'the', 'above', 'low', 'carbon', 'vrp', 'for', 'cavs', 'in', 'addition', 'the', 'differentiation', 'on', 'speed', 'limits', 'in', 'each', 'time', 'p

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['total', 'carbon_emissions', 'minimization', 'in', 'connected', 'and', 'automated', 'vehicle', 'routing', 'problem', 'with', 'speed', 'variables', 'environmental', 'protection', 'and', 'intelligence', 'have', 'become', 'the', 'inevitable', 'development', 'trend', 'of', 'future', 'transportation', 'connected', 'and', 'automated', 'vehicles', 'cavs', 'are', 'expected', 'to', 'be', 'applied', 'in', 'the', 'near_future', 'in', 'this', 'context', 'how', 'to', 'schedule', 'cavs', 'to', 'meet', 'customer', 'demands', 'with', 'carbon_emissions', 'minimization', 'has', 'become', 'new', 'green', 'vehicle', 'routing', 'problem', 'vrp', 'due', 'to', 'the', 'fact', 'that', 'carbon_emissions', 'are', 'tremendously', 'influenced', 'by', 'vehicle', 'speed', 'this', 'paper', 'considers', 'vehicle', 'speed', 'as', 'decision', 'variable', 'in', 'the', 'above', 'low', 'carbon', 'vrp', 'for', 'cavs', 'in', 'addition', 'the', 'differentiation', 'on', 'speed', 'limits', 'in', 'each', 'time', 'period', 'and'

In [9]:
stop_words = stopworddic1
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
#import en

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['total', 'minimization', 'connect', 'automate', 'vehicle', 'route', 'speed', 'variable', 'environmental', 'protection', 'intelligence', 'inevitable', 'development', 'trend', 'future', 'transportation', 'connect', 'automate', 'vehicle', 'cavs', 'expect', 'apply', 'future', 'context', 'schedule', 'cavs', 'meet', 'customer', 'demand', 'minimization', 'green', 'vehicle', 'route', 'vrp', 'tremendously', 'influence', 'vehicle', 'speed', 'paper', 'consider', 'vehicle', 'speed', 'decision', 'variable', 'low', 'carbon', 'vrp', 'cavs', 'addition', 'differentiation', 'speed', 'limit', 'time', 'period', 'type', 'road', 'account', 'study', 'formulate', 'nonlinear', 'programming', 'model', 'outer', 'approximate', 'method', 'linearize', 'propose', 'model', 'hybrid', 'particle_swarm', 'optimization', 'hpso', 'algorithm', 'develop', 'solve', 'extensive', 'numerical', 'experiment', 'conduct', 'validate', 'effectiveness', 'propose', 'model', 'efficiency', 'propose', 'solution', 'method', 'implication',

In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 3), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 2), (42, 3), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 3), (52, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 4), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 5), (70, 2)]]


In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [44]:

from gensim import corpora
dictionary_LDA = corpora.Dictionary(texts)
dictionary_LDA.filter_extremes(no_below=3)
#corpus = [dictionary_LDA.doc2bow(tok) for tok in texts]


In [49]:
data[1]

"LiDAR point's elliptical error model and laser positioning for autonomous vehicles,Automated driving is the development trend of vehicle driving, and laser LiDAR is the high-precision positioning sensor that automated driving relies on most. Laser positioning mainly depends on the relative orientation from the detected environment features to the self-vehicle. However, when there are multiple detected features that carry different orientation information, all the data are redundant. In order to improve the vehicle's positioning accuracy, two laser point evaluation models were put forward. First, based on the plane area analysis method, a spot error model was proposed, and the error distribution with the incidence angle and scanning distance was obtained. Second, the laser point's position approximately obeys the Gaussian distribution, the laser's stereo error ellipsoid model was established, and the laser point's probability volume was calculated through the probability integration. T

In [52]:
from nltk.tokenize import word_tokenize
a= word_tokenize(data[1])

In [53]:
#tokens = word_tokenize(document)

topics = lda_model.show_topics(formatted=True, num_topics=20, num_words=20)
pd.DataFrame([(el[0], round(el[1],2), topics[el[0]][1]) for el in lda_model[dictionary_LDA.doc2bow(a)]], columns=['topic #', 'weight', 'words in topic'])

TypeError: type tuple doesn't define __round__ method

In [47]:
#a

['total',
 'minimization',
 'connect',
 'automate',
 'vehicle',
 'route',
 'speed',
 'variable',
 'environmental',
 'protection',
 'intelligence',
 'inevitable',
 'development',
 'trend',
 'future',
 'transportation',
 'connect',
 'automate',
 'vehicle',
 'cavs',
 'expect',
 'apply',
 'future',
 'context',
 'schedule',
 'cavs',
 'meet',
 'customer',
 'demand',
 'minimization',
 'green',
 'vehicle',
 'route',
 'vrp',
 'tremendously',
 'influence',
 'vehicle',
 'speed',
 'paper',
 'consider',
 'vehicle',
 'speed',
 'decision',
 'variable',
 'low',
 'carbon',
 'vrp',
 'cavs',
 'addition',
 'differentiation',
 'speed',
 'limit',
 'time',
 'period',
 'type',
 'road',
 'account',
 'study',
 'formulate',
 'nonlinear',
 'programming',
 'model',
 'outer',
 'approximate',
 'method',
 'linearize',
 'propose',
 'model',
 'hybrid',
 'particle_swarm',
 'optimization',
 'hpso',
 'algorithm',
 'develop',
 'solve',
 'extensive',
 'numerical',
 'experiment',
 'conduct',
 'validate',
 'effectiveness',
 '

In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.264317202635722

Coherence Score:  0.46560770305758065


In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.218*"traffic" + 0.126*"cavs" + 0.101*"connect" + 0.092*"cav" + '
  '0.064*"flow" + 0.040*"mixed" + 0.028*"connected" + 0.023*"model" + '
  '0.022*"impact" + 0.017*"lane_change"'),
 (1,
  '0.062*"vehicle" + 0.047*"network" + 0.040*"communication" + '
  '0.033*"intersection" + 0.031*"platoon" + 0.028*"time" + 0.027*"traffic" + '
  '0.022*"strategy" + 0.021*"cooperative" + 0.019*"reduce"'),
 (2,
  '0.220*"control" + 0.057*"controller" + 0.036*"vehicle" + 0.027*"track" + '
  '0.025*"design" + 0.022*"lateral" + 0.021*"path" + 0.019*"tracking" + '
  '0.018*"speed" + 0.017*"steering"'),
 (3,
  '0.111*"pedestrian" + 0.102*"target" + 0.056*"platform" + 0.042*"gps" + '
  '0.041*"cross" + 0.029*"output" + 0.027*"crossing" + 0.026*"scene" + '
  '0.020*"machine_learne" + 0.018*"online"'),
 (4,
  '0.048*"service" + 0.047*"transport" + 0.045*"travel" + 0.040*"parking" + '
  '0.036*"share" + 0.034*"transportation" + 0.032*"impact" + 0.030*"mobility" '
  '+ 0.027*"demand" + 0.026*"trip"'),
 (

In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.292597  0.166952       1        1  23.657691
18    -0.285025  0.049225       2        1  22.939855
19    -0.223568 -0.129248       3        1  12.297898
1     -0.235539 -0.086546       4        1   7.333985
2     -0.196826  0.184399       5        1   7.219153
13    -0.078793  0.153574       6        1   4.318267
10    -0.119326 -0.275533       7        1   4.265960
4     -0.083040 -0.257138       8        1   3.410994
12    -0.013116  0.158831       9        1   2.793823
16     0.051658  0.069750      10        1   2.465367
0      0.019554 -0.067890      11        1   1.846491
3      0.094865  0.006250      12        1   1.657000
14     0.112194 -0.039479      13        1   1.396314
15     0.115077  0.022401      14        1   1.278083
7      0.138853 -0.000506      15        1   1.066745
8      0.181394  0.010244      16        1   0.688678
11     0.183904  0.010982      17        1   0.518947
9      0.191890  0.006599      18        1   0.426286
5      0.219310  0.008571      19        1   0.251977
17     0.219130  0.008561      20        1   0.166486, topic_info=                  Term          Freq         Total Category  logprob  loglift
term                                                                         
323            control   7377.000000   7377.000000  Default  30.0000  30.0000
363                car   3266.000000   3266.000000  Default  29.0000  29.0000
84               drive   3768.000000   3768.000000  Default  28.0000  28.0000
54                road   2908.000000   2908.000000  Default  27.0000  27.0000
74          autonomous  12195.000000  12195.000000  Default  26.0000  26.0000
168            traffic   3144.000000   3144.000000  Default  25.0000  25.0000
167             system   7858.000000   7858.000000  Default  24.0000  24.0000
69             vehicle  24216.000000  24216.000000  Default  23.0000  23.0000
121             sensor   2591.000000   2591.000000  Default  22.0000  22.0000
524               lane   1475.000000   1475.000000  Default  21.0000  21.0000
42               model   5350.000000   5350.000000  Default  20.0000  20.0000
6             automate   3628.000000   3628.000000  Default  19.0000  19.0000
79               datum   2230.000000   2230.000000  Default  18.0000  18.0000
324         controller   1895.000000   1895.000000  Default  17.0000  17.0000
434             driver   2298.000000   2298.000000  Default  16.0000  16.0000
309               test   1508.000000   1508.000000  Default  15.0000  15.0000
61                time   3718.000000   3718.000000  Default  14.0000  14.0000
8                 cavs   1083.000000   1083.000000  Default  13.0000  13.0000
51             propose   4123.000000   4123.000000  Default  12.0000  12.0000
40              method   3181.000000   3181.000000  Default  11.0000  11.0000
220             safety   2226.000000   2226.000000  Default  10.0000  10.0000
239                avs   1503.000000   1503.000000  Default   9.0000   9.0000
1054            change   1023.000000   1023.000000  Default   8.0000   8.0000
156            network   1787.000000   1787.000000  Default   7.0000   7.0000
369          detection   1115.000000   1115.000000  Default   6.0000   6.0000
60               study   2907.000000   2907.000000  Default   5.0000   5.0000
75                base   5689.000000   5689.000000  Default   4.0000   4.0000
308         technology   1972.000000   1972.000000  Default   3.0000   3.0000
147              human   1804.000000   1804.000000  Default   2.0000   2.0000
803         navigation   1161.000000   1161.000000  Default   1.0000   1.0000
...                ...           ...           ...      ...      ...      ...
1463   reconfiguration     17.153937     18.169988  Topic20  -3.8082   6.3405
1437      adaptability      7.379299      8.395350  Topic20  -4.6518   6.2690
6925       fuzzy_logic      0.027583   

In [16]:
for i in range(1,15):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    # Compute Perplexity
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.297575736441626

Coherence Score:  0.31666556982550614

Perplexity:  -7.205038941605329

Coherence Score:  0.403356443395379

Perplexity:  -7.1833078964259025

Coherence Score:  0.39793128945467077

Perplexity:  -7.164424294562945

Coherence Score:  0.4617802970104022

Perplexity:  -7.191201745035336

Coherence Score:  0.42754733154932334

Perplexity:  -7.216702254021294

Coherence Score:  0.5001576364936543

Perplexity:  -7.242737951956102

Coherence Score:  0.4631287005746615

Perplexity:  -7.2824973542431914

Coherence Score:  0.48467429111204263

Perplexity:  -7.358405888175963

Coherence Score:  0.513045082111433

Perplexity:  -7.495624827721913

Coherence Score:  0.4964263266972984

Perplexity:  -7.705865368866624

Coherence Score:  0.47558723290502397

Perplexity:  -8.031137505232257

Coherence Score:  0.48116859385381233

Perplexity:  -8.50174748596411

Coherence Score:  0.522953548595681

Perplexity:  -9.021686710646044

Coherence Score:  0.5092348522660758


In [17]:
#import os
#os.environ['MALLET_HOME'] = 'C:/Users/asus/Desktop/mallet-2.0.8'

#mallet_path = './mallet-2.0.8/bin/mallet'

In [32]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


In [33]:
#lda_model.show_topic(0)

In [55]:
# Which document are about topic 14
document_topic.sort_values(14, ascending=False)[1].head(20)

8     0.031342
7     0.000000
3     0.192265
1     0.161874
2     0.000000
4     0.021468
5     0.019640
6     0.051125
9     0.010761
10    0.266920
Name: 1, dtype: float64

In [63]:
from pandas.core.frame import DataFrame
texts1=DataFrame(texts)
topics = [lda_model[corpus[i]] for i in range(len(texts1))]

In [19]:
a=topics[0]
a[0]

[(0, 0.066056974),
 (1, 0.16187352),
 (2, 0.022888262),
 (4, 0.10832651),
 (5, 0.010575332),
 (6, 0.32637447),
 (10, 0.014078437),
 (11, 0.020980071),
 (13, 0.015745368),
 (14, 0.012103397),
 (15, 0.04234971),
 (16, 0.021385178),
 (18, 0.09580477),
 (19, 0.07127355)]

In [151]:
lda_model[corpus[1]]

([(2, 0.016213216),
  (6, 0.26308417),
  (12, 0.040007375),
  (13, 0.11004975),
  (15, 0.018851813),
  (16, 0.32541928),
  (18, 0.13960803),
  (19, 0.06547243)],
 [(6, [19, 1]),
  (9, [6, 19]),
  (17, [18]),
  (40, [6, 13, 12]),
  (42, [6, 2]),
  (51, [6, 2, 12]),
  (65, [18]),
  (69, [18, 6, 19, 2, 1]),
  (71, [13, 12]),
  (72, [18]),
  (73, [2, 16]),
  (74, [18, 6]),
  (75, [6, 18, 13, 12]),
  (76, [6]),
  (77, [18, 6]),
  (78, [13]),
  (79, [13, 18, 19]),
  (80, [18, 6]),
  (81, [12]),
  (82, [16]),
  (83, [6, 1]),
  (84, [19, 6]),
  (85, [19]),
  (86, [18, 16, 2]),
  (87, []),
  (88, []),
  (89, []),
  (90, [6, 13, 18]),
  (91, [16, 13, 2]),
  (92, [6, 18]),
  (93, [6, 19, 18]),
  (94, [15, 18, 19]),
  (95, [6, 13, 18, 12, 2]),
  (96, [16, 13]),
  (97, [12, 18, 6, 19]),
  (98, [6, 18]),
  (99, [18, 2]),
  (100, [16]),
  (101, [6, 13, 18, 19, 1, 2]),
  (102, []),
  (103, [18, 13]),
  (104, [16]),
  (105, [13]),
  (106, [6, 13, 18]),
  (107, [16]),
  (108, [16]),
  (109, [16]),
  (11

In [153]:
lda_model[corpus[1]]

([(2, 0.016234495),
  (6, 0.26307797),
  (12, 0.04000815),
  (13, 0.110046014),
  (15, 0.018851882),
  (16, 0.32541004),
  (18, 0.1396049),
  (19, 0.06547263)],
 [(6, [19, 1]),
  (9, [6, 19]),
  (17, [18]),
  (40, [6, 13, 12]),
  (42, [6, 2]),
  (51, [6, 2, 12]),
  (65, [18]),
  (69, [18, 6, 19, 2, 1]),
  (71, [13, 12]),
  (72, [18]),
  (73, [2, 16]),
  (74, [18, 6]),
  (75, [6, 18, 13, 12]),
  (76, [6]),
  (77, [18, 6]),
  (78, [13]),
  (79, [13, 18, 19]),
  (80, [18, 6]),
  (81, [12]),
  (82, [16]),
  (83, [6, 1]),
  (84, [19, 6]),
  (85, [19]),
  (86, [18, 16, 2]),
  (87, []),
  (88, []),
  (89, []),
  (90, [6, 13, 18]),
  (91, [16, 13, 2]),
  (92, [6, 18]),
  (93, [6, 19, 18]),
  (94, [15, 18, 19]),
  (95, [6, 13, 18, 12, 2]),
  (96, [16, 13]),
  (97, [12, 18, 6, 19]),
  (98, [6, 18]),
  (99, [18, 2]),
  (100, [16]),
  (101, [6, 13, 18, 19, 1, 2]),
  (102, []),
  (103, [18, 13]),
  (104, [16]),
  (105, [13]),
  (106, [6, 13, 18]),
  (107, [16]),
  (108, [16]),
  (109, [16]),
  (110

In [138]:
corpus[1]

[(6, 2),
 (9, 1),
 (17, 1),
 (40, 4),
 (42, 6),
 (51, 3),
 (65, 1),
 (69, 4),
 (71, 3),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 2),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 2),
 (82, 1),
 (83, 2),
 (84, 2),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 6),
 (92, 1),
 (93, 1),
 (94, 3),
 (95, 1),
 (96, 1),
 (97, 3),
 (98, 1),
 (99, 1),
 (100, 1),
 (101, 2),
 (102, 1),
 (103, 1),
 (104, 10),
 (105, 2),
 (106, 1),
 (107, 1),
 (108, 2),
 (109, 1),
 (110, 2),
 (111, 6),
 (112, 1),
 (113, 2),
 (114, 1),
 (115, 1),
 (116, 1),
 (117, 2),
 (118, 1),
 (119, 1),
 (120, 1),
 (121, 1),
 (122, 1),
 (123, 1),
 (124, 2),
 (125, 2),
 (126, 1),
 (127, 1),
 (128, 1),
 (129, 1)]

In [70]:
topics

[([(0, 0.06605673),
   (1, 0.16187441),
   (2, 0.0228853),
   (4, 0.10832576),
   (5, 0.010575332),
   (6, 0.32637474),
   (10, 0.014082707),
   (11, 0.020980071),
   (13, 0.01574537),
   (14, 0.012103398),
   (15, 0.04234971),
   (16, 0.02138573),
   (18, 0.095804766),
   (19, 0.071271524)],
  [(0, [6, 19, 4]),
   (1, [4, 19]),
   (2, [1, 6, 18, 10]),
   (3, [6]),
   (4, [6, 18]),
   (5, [15]),
   (6, [19, 1]),
   (7, [1]),
   (8, [0]),
   (9, [6, 19, 1, 10]),
   (10, [0]),
   (11, [6, 1, 19, 18]),
   (12, [18, 19, 4]),
   (13, [4]),
   (14, [6, 19]),
   (15, [4, 1]),
   (16, [18, 6, 1]),
   (17, [18]),
   (18, [15]),
   (19, [14]),
   (20, [6, 2]),
   (21, [1, 6]),
   (22, [4, 1, 19, 13]),
   (23, [4, 1, 19, 18, 10]),
   (24, [6, 19]),
   (25, [6, 4]),
   (26, [6]),
   (27, [4, 19, 18, 10]),
   (28, [5]),
   (29, []),
   (30, [18]),
   (31, [4, 19, 10]),
   (32, [19]),
   (33, [19, 10, 0]),
   (34, [15]),
   (35, [6, 19, 2, 1]),
   (36, [1]),
   (37, [4]),
   (38, [6, 1, 4, 18, 19, 2

In [108]:
a=[]
for i in range(20):
    a.append(0)
res = pd.DataFrame(a,columns=range(20))


ValueError: Shape of passed values is (1, 20), indices imply (20, 20)

In [119]:
test_1 = pd.DataFrame(np.random.rand(4, 3),
                      index=list('ABCD'), columns=list('123'))  
test_1

1         2         3
A  0.864768  0.935660  0.834665
B  0.772614  0.318033  0.344813
C  0.211382  0.387562  0.291242
D  0.686286  0.314125  0.532111

In [120]:
test_1 = pd.DataFrame(np.random.rand(1, 20),
                      index=list('A'), columns=range(20))  
test_1

0         1         2         3         4        5         6   \
A  0.997032  0.976448  0.504512  0.646328  0.131617  0.38591  0.715915   

         7         8         9         10        11        12        13  \
A  0.710792  0.846965  0.154219  0.902013  0.492972  0.500207  0.299287   

         14        15        16        17        18        19  
A  0.123975  0.399762  0.900512  0.147079  0.581964  0.088913

In [131]:
test_2 = pd.DataFrame()  
for i in range(20):
    test_2.loc["A",i]=0
    

In [132]:
test_2

0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
A  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

    15   16   17   18   19  
A  0.0  0.0  0.0  0.0  0.0

In [58]:
j=0
def topics_document_to_dataframe(topics_document, num_topics):
    global j
    j=j+1
    res = pd.DataFrame(columns=range(num_topics))
    test_2 = pd.DataFrame()  
    for i in range(20):
        test_2.loc[str(j),i]=0
    res1=test_2
    #res.index=0
    #res.ix[]
    for topic_weight in topics_document:
        #print(topic_weight)
        #print(topic_weight[1])
       # print(res)
        #print(res.ix[0, topic_weight[0]])
        res1.loc[str(j),topic_weight[0]] = topic_weight[1]
    return res1
#topics_document_to_dataframe([(0, 0.06605673), (1, 0.16187441), (2, 0.0228853), (4, 0.10832576), (5, 0.010575332), (6, 0.32637474), (10, 0.014082707), (11, 0.020980071), (13, 0.01574537), (14, 0.012103398), (15, 0.04234971), (16, 0.02138573), (18, 0.095804766), (19, 0.071271524)], 20)

In [150]:
 for topic_weight in topics[0]:
        print("*******")
        print(topic_weight[0])

*******
(0, 0.06605673)
*******
(0, [6, 19, 4])
*******
(0, [(4, 0.12925538), (6, 0.7213441), (19, 0.14462875)])


In [21]:
for topics_document in topics[:10]:
    print(topics_document[0])

[(0, 0.066056974), (1, 0.16187352), (2, 0.022888262), (4, 0.10832651), (5, 0.010575332), (6, 0.32637447), (10, 0.014078437), (11, 0.020980071), (13, 0.015745368), (14, 0.012103397), (15, 0.04234971), (16, 0.021385178), (18, 0.09580477), (19, 0.07127355)]
[(2, 0.016236901), (6, 0.2630781), (12, 0.040008496), (13, 0.11004545), (15, 0.0188525), (16, 0.32540914), (18, 0.13960321), (19, 0.065472335)]
[(0, 0.039332308), (1, 0.19226548), (3, 0.0144890705), (4, 0.08975901), (6, 0.14470905), (9, 0.011354249), (12, 0.02616926), (13, 0.015042012), (14, 0.017743865), (16, 0.02465517), (18, 0.29817757), (19, 0.10621208)]
[(0, 0.012569177), (1, 0.021468371), (2, 0.010208254), (3, 0.16153353), (4, 0.05946634), (6, 0.14036599), (7, 0.04595728), (9, 0.0113269035), (10, 0.045276698), (13, 0.038346983), (16, 0.01423132), (18, 0.252223), (19, 0.17600524)]
[(1, 0.019639757), (4, 0.07431256), (6, 0.06604752), (7, 0.042846397), (10, 0.066023886), (11, 0.097999796), (18, 0.30300346), (19, 0.2896575)]
[(0, 0.0

In [64]:
for i in range(10):
    print(topics[i][0])

[(0, 0.066054076), (1, 0.16187343), (2, 0.02289777), (4, 0.108321786), (5, 0.010575331), (6, 0.32636398), (10, 0.014094187), (11, 0.02098007), (13, 0.015745403), (14, 0.012103405), (15, 0.042349707), (16, 0.021383531), (18, 0.09580537), (19, 0.07126747)]
[(2, 0.016222058), (6, 0.26308125), (12, 0.040007565), (13, 0.11004827), (15, 0.018851435), (16, 0.3254153), (18, 0.13960744), (19, 0.06547278)]
[(0, 0.039339412), (1, 0.19224432), (3, 0.014489152), (4, 0.089763254), (6, 0.14472172), (9, 0.011354249), (12, 0.026169322), (13, 0.015041862), (14, 0.017743908), (16, 0.024655186), (18, 0.29817587), (19, 0.10621092)]
[(0, 0.012569186), (1, 0.02147483), (2, 0.010208349), (3, 0.16153356), (4, 0.059463747), (6, 0.1403616), (7, 0.045957178), (9, 0.011326903), (10, 0.045276895), (13, 0.038346365), (16, 0.014231183), (18, 0.25222203), (19, 0.17600732)]
[(1, 0.01963977), (4, 0.074311316), (6, 0.06604742), (7, 0.0428464), (10, 0.066024035), (11, 0.09799979), (18, 0.3030048), (19, 0.28965735)]
[(0, 0

In [24]:

for i in range(10):
    a[i]=topics_document_to_dataframe(i,topics[i][0], 20)
for i in range(10):
    pd.concat(a[i])

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"

In [65]:
document_topic=[]
document_topic = pd.concat([topics_document_to_dataframe(topics_document[0], 20) for topics_document in topics[:100]])

In [66]:
document_topic

0         1         2         3         4         5         6   \
4    0.066054  0.161873  0.022898  0.000000  0.108322  0.010575  0.326364   
5    0.000000  0.000000  0.016222  0.000000  0.000000  0.000000  0.263081   
6    0.039339  0.192244  0.000000  0.014489  0.089763  0.000000  0.144722   
7    0.012569  0.021475  0.010208  0.161534  0.059464  0.000000  0.140362   
8    0.000000  0.019640  0.000000  0.000000  0.074311  0.000000  0.066047   
9    0.033631  0.051130  0.157939  0.000000  0.000000  0.000000  0.508884   
10   0.000000  0.000000  0.026930  0.029075  0.000000  0.000000  0.267375   
11   0.000000  0.031365  0.033501  0.000000  0.012990  0.000000  0.189014   
12   0.000000  0.010754  0.000000  0.000000  0.000000  0.000000  0.362661   
13   0.034202  0.266919  0.000000  0.000000  0.056885  0.000000  0.192778   
14   0.000000  0.057069  0.013179  0.012578  0.000000  0.000000  0.163628   
15   0.075143  0.136051  0.030729  0.031135  0.066616  0.000000  0.157746   
16   0.000000  0.000000  0.034098  0.017219  0.011653  0.000000  0.283185   
17   0.093640  0.087488  0.013346  0.052650  0.075903  0.000000  0.077893   
18   0.000000  0.018260  0.018001  0.025494  0.000000  0.000000  0.302986   
19   0.025173  0.017893  0.020080  0.020271  0.000000  0.000000  0.146116   
20   0.000000  0.012214  0.025292  0.018515  0.000000  0.000000  0.181452   
21   0.015085  0.024230  0.000000  0.000000  0.036053  0.000000  0.152014   
22   0.151011  0.360087  0.025824  0.000000  0.039574  0.000000  0.155068   
23   0.330614  0.070508  0.029158  0.000000  0.036409  0.020041  0.169728   
24   0.021249  0.337631  0.011800  0.000000  0.021812  0.000000  0.144195   
25   0.000000  0.000000  0.024023  0.043064  0.013840  0.000000  0.123928   
26   0.079929  0.023869  0.021281  0.000000  0.000000  0.000000  0.116186   
27   0.045836  0.326306  0.176004  0.000000  0.000000  0.000000  0.183601   
28   0.055255  0.183915  0.172895  0.000000  0.000000  0.000000  0.263898   
29   0.000000  0.154942  0.000000  0.000000  0.473442  0.000000  0.096026   
30   0.000000  0.021119  0.000000  0.000000  0.160208  0.000000  0.037917   
31   0.000000  0.257600  0.010893  0.000000  0.187282  0.000000  0.188482   
32   0.000000  0.032204  0.029019  0.093343  0.016353  0.000000  0.238795   
33   0.000000  0.216606  0.000000  0.013291  0.093024  0.000000  0.140514   
..        ...       ...       ...       ...       ...       ...       ...   
74   0.223203  0.121900  0.071634  0.000000  0.000000  0.000000  0.090482   
75   0.000000  0.000000  0.055781  0.040870  0.017398  0.000000  0.331605   
76   0.011651  0.063828  0.019435  0.000000  0.015810  0.000000  0.303723   
77   0.000000  0.044692  0.010811  0.000000  0.020221  0.000000  0.206776   
78   0.000000  0.022426  0.035778  0.000000  0.019284  0.000000  0.230882   
79   0.080240  0.033224  0.028566  0.010402  0.116758  0.000000  0.236465   
80   0.000000  0.126458  0.039073  0.000000  0.049354  0.000000  0.056509   
81   0.000000  0.000000  0.050277  0.000000  0.085133  0.000000  0.090338   
82   0.089175  0.320268  0.106243  0.000000  0.000000  0.000000  0.242857   
83   0.000000  0.018202  0.024576  0.062622  0.000000  0.000000  0.409826   
84   0.000000  0.012655  0.010277  0.000000  0.000000  0.048006  0.114530   
85   0.000000  0.049922  0.000000  0.000000  0.000000  0.000000  0.153399   
86   0.017843  0.205207  0.157060  0.000000  0.000000  0.000000  0.242859   
87   0.000000  0.084950  0.011315  0.018758  0.140225  0.000000  0.125971   
88   0.000000  0.179779  0.029774  0.019323  0.000000  0.000000  0.083550   
89   0.000000  0.075809  0.525916  0.000000  0.000000  0.000000  0.285081   
90   0.000000  0.085287  0.011074  0.107517  0.173602  0.000000  0.251287   
91   0.170551  0.163204  0.184676  0.000000  0.000000  0.000000  0.036156   
92   0.104502  0.014918  0.071177  0.047792  0.000000  0.000000  0.412197   
93   0.000000  0.000000  0.160638  0.000000  0.077874  0.000000  0.175318   
94   0.

In [68]:
document_topic.sort_values(14, ascending=False)[14].head(20)

63    0.136744
45    0.110661
50    0.097531
48    0.051739
30    0.046562
40    0.042519
59    0.034239
80    0.029746
60    0.028545
11    0.026913
96    0.026819
83    0.025874
92    0.024487
88    0.022851
10    0.021615
53    0.020739
61    0.020716
74    0.020229
19    0.018701
6     0.017744
Name: 14, dtype: float64

In [141]:
len(topics)

5035

In [145]:
for i in range(10):
    topics_document_to_dataframe(i,topics[i], 20)

KeyError: '[0.06605673] not in index'

In [140]:
# Like TF-IDF, create a matrix of topic weighting, with documents as rows and topics as columns
document_topic = pd.concat([topics_document_to_dataframe(topics_document, 20) for topics_document in topics])

KeyError: '[0.06605673] not in index'

In [49]:
lda_model[corpus[20]]

([(0, 0.021248512),
  (1, 0.33763602),
  (2, 0.011799647),
  (4, 0.021812107),
  (6, 0.14419386),
  (7, 0.024444472),
  (16, 0.119300395),
  (18, 0.18704872),
  (19, 0.10028179)],
 [(6, [1, 19]),
  (9, [19, 1, 6]),
  (10, [0]),
  (16, [18, 6, 1]),
  (17, [18]),
  (21, [1, 6]),
  (40, [6]),
  (47, [18, 6, 1]),
  (51, [6, 1]),
  (54, [16]),
  (61, [1, 6]),
  (69, [18, 1, 6, 19]),
  (85, [19]),
  (93, [6, 19, 18, 1, 0]),
  (101, [1, 6, 18, 0, 19]),
  (123, [1, 6]),
  (133, [1, 18, 19]),
  (168, [1, 0, 19]),
  (171, [1, 4]),
  (190, [19, 13]),
  (220, [19, 18, 0]),
  (222, [4]),
  (232, [18, 1, 6]),
  (274, [18, 19]),
  (303, [1, 6, 2]),
  (308, [18, 1, 4, 10]),
  (315, [1, 18, 19, 6]),
  (330, [1, 18, 6]),
  (379, [18, 1, 19, 6]),
  (398, [1, 6, 18, 2, 19]),
  (503, [1]),
  (614, [1, 18]),
  (615, [1]),
  (624, [1]),
  (730, [19, 6]),
  (781, [18]),
  (816, [1]),
  (851, [16]),
  (852, [16]),
  (853, [6]),
  (854, [18]),
  (855, [16]),
  (856, [6]),
  (857, [7, 19]),
  (858, [1, 18, 6]),


In [34]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_lemmatized)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

TypeError: '<' not supported between instances of 'int' and 'tuple'

In [ ]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
#mallet_path = 'C:/Users/asus/Desktop/mallet-2.0.8/bin/mallet' # update this path

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


In [44]:
import os
from gensim.models.wrappers import LdaMallet

os.environ['MALLET_HOME'] = 'F:/mallet-2.0.8'
mallet_path = 'F:/mallet-2.0.8/bin/mallet.bat' # update this path


In [45]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command 'F:/mallet-2.0.8/bin/mallet.bat import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\asus\AppData\Local\Temp\78a8ef_corpus.txt --output C:\Users\asus\AppData\Local\Temp\78a8ef_corpus.mallet' returned non-zero exit status 1.

In [ ]:
#model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)


In [ ]:
#